Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [301]:
import pandas as pd
import numpy as np

df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [302]:
df['gender'].value_counts()

Male      3555
Female    3488
Name: gender, dtype: int64

In [303]:
print("Мужчин:", df.loc[df['gender'] == 'Male'].shape[0]/df.shape[0])
print("Женщин:", 1-df.loc[df['gender'] == 'Male'].shape[0]/df.shape[0])

Мужчин: 0.504756495811444
Женщин: 0.495243504188556


##### 2. Какое количество уникальных значений у поля InternetService?

In [304]:
# можно вывести уникальные количество уникальных значений по всем стобцам с помощью цикла. Это для того, чтобы посмотреть сразу всё

for cat_colname in df.select_dtypes(include='object').columns:
    print(str(cat_colname) + '\n\n' + str(df[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

customerID

2799-TSLAG    1
2929-ERCFZ    1
1904-WAJAA    1
8155-IBNHG    1
2465-BLLEU    1
             ..
2167-FQSTQ    1
2516-XSJKX    1
7036-ZZKBD    1
6532-YLWSI    1
1976-CFOCS    1
Name: customerID, Length: 7043, dtype: int64
****************************************************************************************************

gender

Male      3555
Female    3488
Name: gender, dtype: int64
****************************************************************************************************

Partner

No     3641
Yes    3402
Name: Partner, dtype: int64
****************************************************************************************************

Dependents

No     4933
Yes    2110
Name: Dependents, dtype: int64
****************************************************************************************************

PhoneService

Yes    6361
No      682
Name: PhoneService, dtype: int64
***********************************************************************************************

In [305]:
# Так можно посмотреть уникальные значения отдельного столбца
print(str("InternetService") + '\n\n' + str(df["InternetService"].value_counts()) + '\n' + '*' * 100 + '\n')

InternetService

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64
****************************************************************************************************



In [306]:
# ещё так можно посмореть
df.groupby('InternetService')['customerID'].nunique()

InternetService
DSL            2421
Fiber optic    3096
No             1526
Name: customerID, dtype: int64

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [307]:
df.info() #видно, что столбец TotalCharges имеет тип Object. Необходимо заменить его на числовой тип

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [308]:
#df['TotalCharges'].median() #выведем медиану

In [309]:
#df['TotalCharges'].mean() # cреднее значение

In [310]:
#df['TotalCharges'].std() #стандартное отклонение

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных). 

Ответ: Мы получили исключение, поскольку в колонке приведен строковый тип данных. Необходимо перевести его в числовой float

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [311]:
# df.loc[df['PhoneService'] == 'Yes', 'PhoneService'] = '1'
# можно сделать так

In [312]:
# df.loc[df['PhoneService'] == 'No', 'PhoneService'] = '0'
# и так

In [313]:
df['PhoneService']=df['PhoneService'].replace('Yes', 1)
df['PhoneService']=df['PhoneService'].replace('No', 0)

In [314]:
df['PhoneService']

0       0
1       1
2       1
3       0
4       1
       ..
7038    1
7039    1
7040    0
7041    1
7042    1
Name: PhoneService, Length: 7043, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [266]:
# df.loc[df['TotalCharges'] == '', 'TotalCharges'] = '0' 
# можно заменить так

In [267]:
# df['TotalCharges']=pd.to_numeric(df['TotalCharges'], errors='coerce')
#принудительно преобразуем колонку в числовой тип

In [315]:
df['TotalCharges']=df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges'] = df['TotalCharges'].astype('float32')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [316]:
df['TotalCharges'].median() #выведем медиану

1394.550048828125

In [317]:
df['TotalCharges'].mean() # cреднее значение

2279.732177734375

In [318]:
df['TotalCharges'].std() #стандартное отклонение

2266.79443359375

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [319]:
df['Churn']

0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7038     No
7039     No
7040     No
7041    Yes
7042     No
Name: Churn, Length: 7043, dtype: object

In [320]:
df.loc[df['Churn'] == 'Yes', 'Churn'] = '1'

In [321]:
df.loc[df['Churn'] == 'No', 'Churn'] = '0'

In [322]:
df['Churn']=pd.to_numeric(df['Churn'], errors='coerce')

In [323]:
df['Churn']

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [324]:
for colname in ['StreamingMovies', 'StreamingTV', 'TechSupport']:
    df[colname]=df[colname].replace('No', 0)
    df[colname]=df[colname].replace('Yes', 1)
    df[colname]=df[colname].replace('No internet service', 0)

##### 8. Заполните пропуски в поле PhoneService значением 0

In [326]:
len(df.loc[df['PhoneService'].isnull(), :].index) #пропусков нет

0

In [328]:
df['PhoneService'] = df['PhoneService'].fillna(0) # на всякий случай

In [329]:
df.loc[df['PhoneService'].isnull(), 'PhoneService'] = 0 #y

In [330]:
df['PhoneService']=pd.to_numeric(df['PhoneService'], errors='coerce')

In [331]:
len(df.loc[df['PhoneService'].isnull(), :].index)

0

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [332]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   int64  
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   int64  
 13  StreamingTV       7043 non-null   int64  
 14  StreamingMovies   7043 non-null   int64  
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [333]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
df=df[columns]

In [334]:
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.850000,0,0,0,0
1,Male,34,1,1889.500000,0,0,0,0
2,Male,2,1,108.150002,0,0,0,1


In [335]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   gender           7043 non-null   object 
 1   tenure           7043 non-null   int64  
 2   PhoneService     7043 non-null   int64  
 3   TotalCharges     7043 non-null   float32
 4   StreamingMovies  7043 non-null   int64  
 5   StreamingTV      7043 non-null   int64  
 6   TechSupport      7043 non-null   int64  
 7   Churn            7043 non-null   int64  
dtypes: float32(1), int64(6), object(1)
memory usage: 412.8+ KB


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [336]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

X_train, X_test, y_train, y_test = train_test_split(df, df['Churn'], random_state=0)


##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [337]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

#### Как вызвать pipeline я разобрался. Но я не очень понимаю зачем этот pipeline нужен и в чем его физический смысл. 
Без него можно обойтись. Для чего он нужен?

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [338]:
gender.fit_transform(X_train)

,gender_Female,gender_Male
3296,1,0
6397,0,1
6043,0,1
5309,1,0
3000,0,1
...,...,...
4931,0,1
3264,1,0
1653,0,1
2607,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [339]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [340]:
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

Объединение всех "кубиков" очень легко сделать таким образом

In [341]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [342]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [343]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [344]:
preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.22, 0.13, 0.25, 0.02, 0.3 , 0.25, 0.  , 1.  , 0.01, 0.06])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [346]:
from sklearn.metrics import roc_auc_score, log_loss
print(f'roc_auc = {roc_auc_score(y_test, preds)}')
print(f'log_loss = {log_loss(y_test, preds)}')

roc_auc = 0.7787583156675664
log_loss = 0.80510091130902


In [349]:
!pip install dill

  Created wheel for dill: filename=dill-0.3.2-py3-none-any.whl size=78977 sha256=15237210c848d0dc2a14f5e376513c979a7802ffd7545672e374715f77e1d174
  Stored in directory: c:\users\vld85\appdata\local\pip\cache\wheels\93\7f\7d\78ec535a4340ef2696aad8b17fe8bb063d56301bd62881b069
Successfully built dill


### Сохраним наш пайплайн

In [350]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)

# расскажите, пожалуйста, поподробнее про pipeline. Что это такое и зачем это нужно